In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import cross_val_score, TimeSeriesSplit
from datetime import timedelta

import warnings
warnings.filterwarnings('ignore')

from copy import deepcopy

from helpers_preprocessing import *
from helpers_postprocessing import *
from helpers_clustering import *
from helpers_plot import *
from helpers_modelweek import *
from helpers_linear_regression import *
from building_routine import analyze_building

%matplotlib inline
%load_ext autoreload
%autoreload 2

## Import + process/clean building data 

Small preprocess: before going down the code of the clustering, I re-download the data to have the names of the stations....

In [ ]:
dat = pd.read_csv("data/191126_E3M_LESO_GMOSRawValuesWithRanking.csv", header=0,encoding="ISO-8859-1")

In [ ]:
dat.head(5)

In [ ]:
## Question to Dan: what is this?

In [ ]:
# Load the raw dataset
data_path = "data/data.csv"
data_raw = load_data(data_path)

In [ ]:
# extract the names of the buildings (change "Locations" to 0 to make the rest of the code work...)
locations = [0] + list(dat.iloc[2])[1:]

# Add at the end of teh rows the location names to keep track of them...
data_raw.columns = locations

In [ ]:
data_raw

Preprocess...

In [ ]:
# Import the columns indices that we need to use from external file...
data_col = pd.read_csv('data/columns.csv', header=None)
# ...and select only columns related to energy consumption
# column 0 correspond to measurement time, all the others are single building measurenets
energy_consumption_columns = np.concatenate(([0], np.hstack(data_col.values)))

# Format raw dataset by:
# 1) selecting only columns related to energy consumption 
# 2) casting index to datetime
# 3) removing wrong measurements (duplicated rows, missing time records)
# 4) dropping last 6 days in order to have complete weeks
data_with_NaNs = format_data(data_raw, energy_consumption_columns)

# Discard columns which contains more than max_NaNs, otherwise fill empty values with pd.fillna()
# selected_columns is used to link results to the corresponding building in the raw dataset
max_NaNs = 100
data_no_NaNs, selected_columns = NaN_handling(data_with_NaNs, max_NaNs, energy_consumption_columns)

# Discard columns representing small buildings (setting the threshold for the mean energy consupmtion)
minimum_mean_energy_consumption = 0
data_no_small, selected_columns = remove_small_buildings(data_no_NaNs, minimum_mean_energy_consumption, selected_columns)


# ------- KEEP TRACK OF THE LOCATIONS OF EACH BUILDING, in the same order
selected_buildings = list(data_no_small.columns)


# Rename columns as "building_" + N, where N in [0, #columns-1]
data_renumbered = renumber_columns(data_no_small)

# Rename the processed dataset
data = data_renumbered

data.head(3)

In [ ]:
print(selected_buildings)

## Building weekly anomaly detection: Fourier + PCA + DBSCAN clustering

We want to look at multiple buildings (there are 23x7 weeks data - 23 months from january 2018 until mid november 2019) and:

- Apply Fourier (features) + PCA + DBSCAN
- Detect : Normal (1st cluster) // Anormal (2nd cluster) // big outliers (outliers)
- Check the time series to see if the clusters actually make sense! 

In [ ]:
import seaborn as sns

sns.set(style="ticks", font_scale=1, context="talk")
sns.set_style("white", {'axes.grid' : False})
plt.rcParams['figure.figsize'] = (22, 29)

In [ ]:
def format_energy_plot(y_min, y_max):
    """
    Utility to format all time series plots
    :param double y_min: minimum y values displayed
    :param double y_max: maximum y values displayed
    """
    plt.gca().set_ylim([y_min, 1.2*y_max])  # TODO: fix y_max to add also legend
    plt.xticks(rotation=20)
    plt.gca().set_ylabel('[kWh]')
    plt.gca().set_xlabel('')
    plt.grid('True')

#### PLOT!!

Make nice plots for each building, while keeping in track the normal and anormal weeks... We have 119 buildings. 

In [ ]:
building_WeeksTypes_list = []

# for each building
for building_number in range(1,120):
    
    ##################################################################################################################################
    # ----------------------------------------------------- Pre-processing --------------------------------- 
    ##################################################################################################################################


    # ----------- create dataframe for time series ---------------------------
    # All the time series in our dataset have 15 minutes resolution, resulting in 96 measurements per day.
    data_per_day = 96
    data_per_week = data_per_day*7

    energy_consumptions, total_weeks = create_single_building_dataframe(data, building_number, data_per_week)
    #energy_consumptions.head(10)

    ##################################################################################################################################
    # ----------------------------------------------------- PCA + DBSCAN --------------------------------- 
    ##################################################################################################################################

    # ----------- Fourier + PCA + DBSCAN ---------------------------
    # Value used to define the epsilon parameter of DBSCAN, obtained
    # through grid search (see notebook grid_search_DBSCAN)
    first_singular_value_multiplier = 0.012
    # Analysis performed in 2 dimension
    principal_components_number = 2
    # FT + PCA on selected building
    PCA_weeks, S = compressed_week_representation(energy_consumptions, 
                                              principal_components_number, 
                                              data_per_week, 
                                              total_weeks)
    # Run DBSCAN to identify the clusters in the space defined by the Principal Components
    epsilon = S[0]*first_singular_value_multiplier
    clustering = DBSCAN(eps=epsilon, min_samples=9).fit(PCA_weeks.T)
    # Assign each week to a cluster and reorder clusters based on their cardinality
    # the bigger one contains normal weeks and is plotted in blue; the remaining clusters and outliers (black)
    # are the atypical weeks
    ordered_clusters, outliers = extract_clusters(clustering)
    
    
    
    
    ##### extract classes of weeks
    normal_weeks, atypical_weeks,outliers_weeks = weeks_clustering(ordered_clusters,outliers,total_weeks)
    
    building_WeeksTypes_list.append((normal_weeks, atypical_weeks,outliers_weeks))
    
    
    ##################################################################################################################################
    # ----------------------------------------------------- PLOTS --------------------------------- 
    ##################################################################################################################################


    # actual location
    
    loc = selected_buildings[building_number-1]
    
    fig = plt.figure()
    # time series values to set up the subplots
    data_per_group = data_per_week*4
    group_range = [0, int(total_weeks/4)]
    week_indices = np.arange(0, total_weeks)

    # -------------------- CLUSTER points ----------------------------------------
    max_cluster_index = np.max(clustering.labels_)

    plt.subplot(int(group_range[1]/3) + 1 ,1, 1)

    for cluster_index in range(0, max_cluster_index+1):
        plt.scatter(PCA_weeks[0, ordered_clusters[cluster_index]], PCA_weeks[1, ordered_clusters[cluster_index]])
    plt.scatter(PCA_weeks[0, outliers], PCA_weeks[1, outliers], color='black')
    if building_number != None:
        plt.title('Building ' + str(building_number) + " (" + loc + ") ",fontsize=30)
    plt.gca().set_xlabel('PC1')
    plt.gca().set_ylabel('PC2')
    plt.tick_params(
        axis='x',          # changes apply to the x-axis
        which='both',      # both major and minor ticks are affected
        bottom=False,      # ticks along the bottom edge are off
        top=False,         # ticks along the top edge are off
        labelbottom=False)
    #plt.show()

    # -------------------- Time series ----------------------------------------

    #plt.figure(figsize=(20, 4))
    for group_index in range(group_range[0], group_range[1]):

        plt.subplot(int(group_range[1]/3)+1,3,group_index + 4)
        for local_week_index in range(0, 4):
            global_week_index = group_index*4 + local_week_index
            # Look for the current week in every cluster (plot with relative color)
            found_in_cluster = False
            for cluster_index in range(0, len(ordered_clusters)):
                if (global_week_index in week_indices[ordered_clusters[cluster_index]]):
                    week_color = 'C' + str(cluster_index)
                    energy_consumptions.loc[energy_consumptions['week'] == global_week_index, 'consumption']\
                                       .plot(grid='True',rot=45, color=week_color)
                    found_in_cluster = True
            # If week not found then it's outlier (plot in black)
            if not found_in_cluster:
                energy_consumptions.loc[energy_consumptions['week'] == global_week_index, 'consumption']\
                                   .plot(grid='True', rot=45, color='black')
        plt.title('4Weeks Group ' + str(group_index))
    
        format_energy_plot(energy_consumptions.consumption.min(),energy_consumptions.consumption.max())
        
    #plt.show()
    plt.subplots_adjust(hspace=0.9)
    #plt.tight_layout()

    # save
    fig.savefig("".join(["MigrosBuildings_Weeks_DBSCAN_and_TimeSeries/Weeks_Points_and_TimeSeries_building_",str(building_number),".pdf"]), dpi=300, box_inches= "tight")

In [ ]:
[el for el in range(119,120)]

#### We extracted the normal, anormal, and outlier weeks for each building, while plotting.  (NOTE that the outliers are also listed in the anormal points)

Now we can form it in a nice table, giving all weeks, specifying if each week is normal or not... Normal:1 / Anormal: 0 / Outlier:-1 / 

In [ ]:
week_building_list = []

for building_weeks in building_WeeksTypes_list:
    # types of weeks    
    normal, anormal, outliers = set(building_weeks[0]),  set(building_weeks[1]),  set(building_weeks[2])
    #print(len(normal)+len(anormal)+len(outliers))
    week_list = []
    # we say: 
    for week_i in range(98):
        if week_i in normal:
            week_list.append(1)
        else:
            if week_i in outliers:
                week_list.append(-1)
            else:
                week_list.append(0)

    week_building_list.append(week_list)

In [ ]:
# nice table
building_weeks_MIGROS = pd.DataFrame(np.array(week_building_list))
# tu put time in index, buildings in columns
building_weeks_MIGROS = building_weeks_MIGROS.transpose()

# columns = locations
building_weeks_MIGROS.columns = selected_buildings
# index = week index

# Extract week end data 
loul = data.resample("1W", how="sum")
WEEKS_END = loul.index
building_weeks_MIGROS.index = [el + timedelta(days=-6) for el in WEEKS_END] 
building_weeks_MIGROS.index.name = "Week_start"

building_weeks_MIGROS

Export the data in Excel

In [ ]:
building_weeks_MIGROS.to_excel("Anomaly_detection_Weeks_Migros_buildings.xlsx",
             sheet_name='Sheet_name_1') 

Plot a mesh, just for fun. 

In [ ]:
plt.figure()
plt.imshow(building_weeks_MIGROS)
plt.show()

NOTE: some buildings have a lot of outliers. 
Reasons? 

It's maybe because the anomalies can be like a missing day but also much higher or small values in the demand --> Seasonality? 
But then why doesnt it create another clusters if there are a lot of similar outliers ???